In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import socket
from socket import *
import time
import os


learning_rate = 0.001



trainset = torchvision.datasets.CIFAR10(root='./data', train=True , download=True, transform=transforms.ToTensor())
testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transforms.ToTensor())  

train_loader = torch.utils.data.DataLoader(trainset, batch_size=20,shuffle=True)                                       
testloader = torch.utils.data.DataLoader(testset, batch_size=4,shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')


class model(nn.Module):
    def __init__(self, num_classes=10):
        super(model, self).__init__()
        self.conv1 = torch.nn.Sequential()
        self.conv1.add_module("conv1", nn.Conv2d(3, 32, 3, 1, 2))
        self.conv1.add_module("relu1", torch.nn.ReLU())
        self.conv2 = torch.nn.Sequential()
        self.conv2.add_module("conv2", nn.Conv2d(32, 64, 3, 1, 2))
        self.conv2.add_module("relu2", torch.nn.ReLU())
        self.conv2.add_module("pool2", torch.nn.MaxPool2d(2,2))
        self.drop1 = torch.nn.Dropout(0.25)

        self.conv3 = torch.nn.Sequential()
        self.conv3.add_module("conv3", nn.Conv2d(64, 128, 3, 1, 2))
        self.conv3.add_module("relu3", torch.nn.ReLU())
        self.conv3.add_module("pool3", torch.nn.MaxPool2d(2))

        self.conv4 = torch.nn.Sequential()
        self.conv4.add_module("conv4", nn.Conv2d(128, 128, 3, 1, 2, padding_mode='SAME'))
        self.conv4.add_module("relu4", torch.nn.ReLU())
        self.conv4.add_module("pool4", torch.nn.MaxPool2d(2))
        self.drop2 = torch.nn.Dropout(0.25)

        self.dense1 = torch.nn.Sequential()
        self.dense1.add_module("fc1", nn.Linear(128 * 6 * 6, 1500))
        self.dense2 = torch.nn.Sequential()
        self.drop3 = torch.nn.Dropout(0.5)

        self.dense2.add_module("fc2", nn.Linear(1500, num_classes))
        
    def forward(self,x):
        conv1 = self.conv1(x)
        conv2 = self.conv2(conv1)
        conv2 = self.drop1(conv2)
        conv3 = self.conv3(conv2)
        conv4 = self.conv4(conv3)
        conv4 = self.drop2(conv4)
        conv4 = conv4.reshape(conv4.size(0), -1)
        # print(conv2.shape)
        fc1 = self.dense1(conv4)
        fc1 = self.drop3(fc1)
        fc2 = self.dense2(fc1)
        return fc2


Files already downloaded and verified
Files already downloaded and verified


In [2]:
classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Current CUDA device: '+str(device))
print('Current CUDA device name: '+torch.cuda.get_device_name(device))

Current CUDA device: cuda:0
Current CUDA device name: NVIDIA GeForce RTX 2080 Ti


In [3]:
net = model().to(device)

In [5]:
num_epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate)
for h in range(1,num_epochs):
    print("#epoch")
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device) # send inputs and targets to GPU 
        optimizer.zero_grad()
        # forward + backward
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()


#epoch
#epoch
#epoch
#epoch
#epoch
#epoch
#epoch
#epoch
#epoch


In [8]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels =  data[0].to(device), data[1].to(device)
        outputs = net(images).to(device)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


In [9]:

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))




Accuracy of plane : 52 %
Accuracy of   car : 76 %
Accuracy of  bird : 37 %
Accuracy of   cat : 43 %
Accuracy of  deer : 72 %
Accuracy of   dog : 52 %
Accuracy of  frog : 71 %
Accuracy of horse : 70 %
Accuracy of  ship : 85 %
Accuracy of truck : 79 %
